# 图像识别分类项目 - 学习笔记
 
## 项目概述
 
本项目致力于开发一个高效的图像识别分类系统，采用了著名的CIFAR-10数据集进行训练和测试。CIFAR-10数据集包含了10个类别的60000张32x32彩色图像，涵盖了飞机、汽车、鸟类、猫、鹿、狗、蛙类、马、船和卡车这10样常见物体。利用PyTorch深度学习框架，构建一个基于卷积神经网络（CNN）的图像分类模型，并通过一系列优化措施提升了模型的性能。

## Requirements

要运行本项目，需要满足以下软件和库的要求：

### 基本要求

Python：建议使用Python 3.8或更高版本，以确保与项目中使用的所有库的兼容性。

PyTorch：一个开源的深度学习库，提供了丰富的神经网络组件和GPU加速功能。本项目依赖于PyTorch及其相关库torchvision。

### 库依赖

PyTorch：版本建议为1.8.0或更高，以支持项目中使用的神经网络架构和训练流程。

torchvision：PyTorch的图像和视频处理库，版本建议为0.9.0或更高，用于数据增强和加载CIFAR-10数据集。

TensorBoard：一个用于可视化深度学习模型训练过程的工具，版本建议为2.5.0或更高。

pillow（PIL Fork）：一个图像处理库，用于图像的预处理和增强。

## 项目结构与内容
 
### 数据准备
 
CIFAR-10数据集从官方渠道下载，并被划分为训练集和测试集。训练集包含50000张图像，用于模型的训练；测试集包含10000张图像，用于模型的评估。
 
### 数据预处理
 
在数据预处理阶段，对图像进行归一化处理，使其像素值在0到1之间，同时保持了图像的原始尺寸（32x32）。此外，还使用了PyTorch的`torchvision.transforms`模块进行数据增强，包括随机水平翻转和随机裁剪，以增加模型的泛化能力。具体的数据增强代码如下所示：

In [ ]:
import torchvision.transforms as transforms
 
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

使用上述train_transform对训练集和测试集进行处理：

In [ ]:
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# 预处理数据集
train_data = datasets.CIFAR10("./dataset", train=True, transform=train_transform, download=True)
test_data = datasets.CIFAR10("./dataset", train=False, transform=train_transform, download=True)

# 训练集和测试集长度
train_data_size = len(train_data)
test_data_size = len(test_data)
print(f"训练数据集长度为：{train_data_size}")
print(f"测试数据集长度为：{test_data_size}")

# 利用DataLoader加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

## 模型构建与优化、训练

In [ ]:
import torch
from torch import nn
import torchvision
from torch.optim.lr_scheduler import StepLR

# 定义训练的设备
device = torch.device("cuda")

# 搭建神经网络
class VisionNet(nn.Module):
    def __init__(self):
        super(VisionNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 5, 1, 2),
            nn.BatchNorm2d(64),  # 添加批量归一化层
            nn.ReLU(),  # 非线性变换激活函数
            nn.MaxPool2d(2),

            nn.Conv2d(64, 64, 5, 1, 2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 5, 1, 2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 128, 5, 1, 2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(128 * 2 * 2, 128),
            nn.ReLU(),
            nn.Dropout(0.5),  # 添加Dropout层，丢弃率为0.5
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x

model = VisionNet()
model.to(device)

# 简单验证模型结构
if __name__ == '__main__':
    input = torch.ones((64, 3, 32, 32))
    output = model(input)
    print(output.shape)

### 网络架构

设计一个包含多个卷积层、池化层和全连接层的CNN模型。卷积层用于提取图像特征，池化层用于降低特征图的维度，全连接层用于分类。网络架构的具体细节可能根据实验和调优有所不同，但通常包括以下几个部分：
    
    卷积层：使用多个卷积核提取图像特征。
    
    池化层：如最大池化层，用于降低特征图的尺寸。
    
    全连接层：将特征图映射到类别空间。

### 激活函数

在每个卷积层和全连接层后，使用ReLU激活函数，以引入非线性特性。ReLU函数的形式为f(x) = max(0, x)，它有助于解决梯度消失问题，并加速模型的收敛。

### 损失函数与优化器

在训练过程中，选择交叉熵损失函数作为模型的损失函数。交叉熵损失函数常用于多分类问题，它能够衡量模型输出与真实标签之间的差异。SGD优化器以较小的学习率逐步更新模型参数，有助于找到全局最优解。

损失函数和优化器的代码如下所示：

In [ ]:
import torch.optim as optim

# 损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

# 优化器
learning_rate = 1e-1
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

### 学习率调度器
 
为了进一步提高模型的性能，采用学习率衰减策略。通过`torch.optim.lr_scheduler.StepLR`，设置每10个epoch学习率减半。

In [ ]:
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

## 训练与评估循环

接下来，设置训练与评估的循环。在每个epoch中，模型首先在训练集上进行训练，然后在测试集上进行评估。

### 训练步骤

在训练步骤中，模型被设置为训练模式（model.train()），以确保Dropout层和Batch Normalization层按预期工作。然后，使用DataLoader迭代训练数据，每次取出一个batch的数据进行前向传播、计算损失、反向传播和优化。同时，添加tensorboard，以可视化训练过程。

In [ ]:
# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0

# 添加tensorboard可视化训练过程
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./new_logs_train")
cumulative_test_loss = 0
cumulative_accuracy = 0
num_test_batches = len(test_dataloader)

# 监控时间
import time
start_time = time.time()

# 训练的轮数
epoch = 50

### 评估步骤

在评估步骤中，模型被设置为评估模式（model.eval()），以关闭Dropout层和Batch Normalization层的训练特性。然后，使用DataLoader迭代测试数据，计算模型在测试集上的损失和准确率。

In [ ]:
for i in range(epoch):
    print(f"\n————————————第{i+1}轮训练开始————————————")

    # 训练步骤开始
    model.train()
    total_train_loss = 0
    total_train_correct = 0
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = model(imgs)
        loss = loss_fn(outputs, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(end_time - start_time)
            print(f"训练次数：{total_train_step}, Loss:{loss.item()}")
            writer.add_scalar("train_loss", loss.item(), total_train_step)

        # 计算正确预测的样本数
        correct = (outputs.argmax(1) == targets).sum().item()
        total_train_correct += correct

        # 累加训练损失
        total_train_loss += loss.item()

    # 计算训练集上的平均Loss和正确率
    avg_train_loss = total_train_loss / len(train_dataloader)
    avg_train_accuracy = total_train_correct / len(train_dataloader.dataset)

    # 记录训练集上的平均Loss和正确率到TensorBoard
    writer.add_scalar("train_loss_avg", avg_train_loss, total_train_step)
    writer.add_scalar("train_accuracy_avg", avg_train_accuracy, total_train_step)

    # 更新学习率
    scheduler.step()

    # 测试步骤开始
    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)

            # 累加每个批次的Loss和正确率
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy +=accuracy

        # 增加测试步骤计数器
        total_test_step += 1
        # 计算测试集的平均Loss和正确率
        avg_test_loss = total_test_loss / len(test_dataloader)
        avg_accuracy = total_accuracy / len(test_dataloader.dataset)
        # 记录测试集的平均Loss和正确率到TensorBoard
        writer.add_scalar("test_loss_avg", avg_test_loss, total_test_step)
        writer.add_scalar("test_accuracy_avg", avg_accuracy, total_test_step)
        # 重置累计值，为下一个epoch做准备
        cumulative_test_loss = 0
        cumulative_accuracy = 0
        # 打印整体测试集上的Loss和正确率
        print(f"\n整体测试集上的平均Loss: {avg_test_loss}")
        print(f"整体测试集上的平均正确率: {avg_accuracy}")

    # 保存模型
    torch.save(model.state_dict(), f"./new_model_i/model_{i+1}.pth")
    print("模型已保存")

### 模型保存

在每个epoch结束后，将训练好的模型保存到磁盘上，以便将来进行进一步的分析或部署。

### TensorBoard记录

为了可视化训练过程中的损失和准确率，使用TensorBoard。在训练结束后，关闭TensorBoard的SummaryWriter。

In [ ]:
writer.close()

在终端输入以下指令打开TensorBoard查看logs

tensorboard --logdir=new_logs_train

## 模型应用

使用训练好的VisionNet模型对图像进行识别分类。

首先，设置图像存放的目录和模型文件的路径。
接着，定义图像转换流程，包括调整图像大小为32x32像素，并将其转换为张量格式以适应模型输入。
然后，加载VisionNet模型，并通过load_state_dict方法导入之前保存的模型参数。将模型设置为评估模式后，定义包含10个类别名称的列表，以便将模型输出的类别索引映射到具体的类别名称。

随后，使用glob模块遍历指定目录下的所有图像文件。
对于每张图像，先打开并转换为RGB格式，再应用之前定义的转换流程，并添加批次维度以符合模型输入要求。
在关闭梯度计算的情况下，将图像输入模型进行前向传播，得到预测输出。
最后，通过argmax函数获取预测类别索引，并打印出图像路径和预测的类别名称。

In [ ]:
import os
import glob
from PIL import Image

# 设置图片目录和模型路径
image_dir = "./test_set/"
model_path = "./new_model_i/model_50.pth"

# 定义图片转换
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

# 加载网络模型
model = VisionNet()
model.load_state_dict(torch.load(model_path))
model.eval()

# 类别名称列表
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

# 图片格式列表，根据需要添加或删除格式
image_formats = ["*.png", "*.jpg", "*.jpeg"]

# 获取所有匹配的图片文件路径
image_files = []
for fmt in image_formats:
    image_files.extend(glob.glob(os.path.join(image_dir, fmt)))

# 遍历每张图片
for image_path in image_files:
    image = Image.open(image_path)
    image = image.convert('RGB')
    
    # 应用转换
    image = transform(image)
    image = image.unsqueeze(0)  # 添加批次维度
    
    with torch.no_grad():
        output = model(image)
    
    # 打印预测的类别
    predicted_class = output.argmax(1).item()
    predicted_class_name = class_names[predicted_class]
    print(f"Image: {image_path}, Predicted class: {predicted_class_name}")

## 总结

通过本项目，深入了解PyTorch深度学习框架的使用，掌握CIFAR-10数据集的加载和预处理方法。学会如何设计和优化CNN模型，包括数据增强、激活函数的选择、损失函数与优化器的使用、学习率调整以及Dropout等优化技巧。此外，学会如何使用TensorBoard进行训练过程的可视化。最终，构建一个能够相对准确地对上述的10个类别的图像进行识别分类的模型。